In [1]:
# Se importan las librerías
import pandas as pd
from neo4j import GraphDatabase
import time

In [2]:
# Se carga el csv con el preprocesado de la parte de Lucas
# El csv es del 20/01/2022
df = pd.read_csv('total_procesed.csv')
df.head()

,_id,fuente,titulo,autores,abstract,clase_primaria,pag_espec,n_citaciones,citado,abstract_limpio,keywords
0,{'ObjectId': '639d9f0d6d67832e75895c46'},arxiv,['Reinforcement Learning through Asynchronous ...,"['Mohammad Babaeizadeh', 'Iuri Frosio', 'Steph...","['\n ', "" We introduce a hybrid CPU/GPU ...",['Machine Learning (cs.LG)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 222'],['/scholar?cites=8757115672331028243&as_sdt=20...,"['\n ', "" We introduce a hybrid CPU/GPU ...","[array([], dtype=float64), array(['computation..."
1,{'ObjectId': '639d9f0d6d67832e75895c47'},arxiv,['Altruistic Maneuver Planning for Cooperative...,"['Behrad Toghi', 'Rodolfo Valiente', 'Dorsa Sa...","['\n ', "" With the adoption of autonomou...",['Robotics (cs.RO)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 11'],['/scholar?cites=1543039800876013962&as_sdt=20...,"['\n ', "" With the adoption of autonomou...","[array([], dtype=float64), array(['autonomous ..."
2,{'ObjectId': '639d9f0e6d67832e75895c48'},arxiv,['Towards Understanding Asynchronous Advantage...,"['Han Shen', 'Kaiqing Zhang', 'Mingyi Hong', '...","['\n ', ' Asynchronous and parallel impl...",['Machine Learning (cs.LG)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 10'],['/scholar?cites=17816434878891582694&as_sdt=2...,"['\n ', ' Asynchronous and parallel impl...","[array([], dtype=float64), array(['##ch', '##y..."
3,{'ObjectId': '639d9f0e6d67832e75895c49'},arxiv,['The Advantage Regret-Matching Actor-Critic'],"['Audrūnas Gruslys', 'Marc Lanctot', 'Rémi Mun...","['\n ', "" Regret minimization has played...",['Artificial Intelligence (cs.AI)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 14'],['/scholar?cites=13622395690856603026&as_sdt=2...,"['\n ', "" Regret minimization has played...","[array([], dtype=float64), array(['##ability',..."
4,{'ObjectId': '639d9f0e6d67832e75895c4a'},arxiv,['Actor-Critic Sequence Training for Image Cap...,"['Li Zhang', 'Flood Sung', 'Feng Liu', 'Tao Xi...","['\n ', ' Generating natural language de...",['Computer Vision and Pattern Recognition (cs....,https://scholar.google.com/scholar?q=advantage...,['Citado por 111'],['/scholar?cites=16998376404483706528&as_sdt=2...,"['\n ', ' Generating natural language de...","[array([], dtype=float64), array(['captioning ..."


In [9]:
df.value_counts('clase_primaria')

clase_primaria
[]                                                                                                                                       9
['Machine Learning (cs.LG)']                                                                                                             5
['Artificial Intelligence (cs.AI)']                                                                                                      2
['Robotics (cs.RO)']                                                                                                                     2
['Actor critic', 'deep Q network(DQN)', 'inverted pendulum', 'PID', 'reinforcement learning']                                            1
['Big data', 'health informatics', 'machine learning', 'mental health']                                                                  1
['Computer Vision and Pattern Recognition (cs.CV)']                                                                                      1
['Deep learn

In [11]:
df.value_counts('autores')

autores
['Chengmin Zhou', 'Bingding Huang', 'Pasi Fränti']                                                                                                                                                                                                       2
['Adrian B. R. Shatte', 'Delyse M. Hutchinson', 'Samantha J. Teague']                                                                                                                                                                                    1
['Raafat Shalaby', 'Mohammad El-Hossainy', 'Tarek A. Mahmoud']                                                                                                                                                                                           1
['Zipiao Zhao', 'Yikai Liu', 'Yongli Zhao', 'Yajie Li', 'Sabidur Rahman', 'Dahai Han', 'Jie Zhang']                                                                                                                                            

### Conectar a Neo4j Desktop

In [3]:
# Se crea una clase en la que se esctablece la conexión entre la máquina local donde
# se encuentra el código de python y la base de datos de Neo4j Desktop.
# Le indicamos la URL, el usuario y la contraseña para completar la conexión
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
        
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="usermate",              
                       pwd="glass")

### Creación BBDD

In [4]:
df.head()

,_id,fuente,titulo,autores,abstract,clase_primaria,pag_espec,n_citaciones,citado,abstract_limpio,keywords
0,{'ObjectId': '639d9f0d6d67832e75895c46'},arxiv,['Reinforcement Learning through Asynchronous ...,"['Mohammad Babaeizadeh', 'Iuri Frosio', 'Steph...","['\n ', "" We introduce a hybrid CPU/GPU ...",['Machine Learning (cs.LG)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 222'],['/scholar?cites=8757115672331028243&as_sdt=20...,"['\n ', "" We introduce a hybrid CPU/GPU ...","[array([], dtype=float64), array(['computation..."
1,{'ObjectId': '639d9f0d6d67832e75895c47'},arxiv,['Altruistic Maneuver Planning for Cooperative...,"['Behrad Toghi', 'Rodolfo Valiente', 'Dorsa Sa...","['\n ', "" With the adoption of autonomou...",['Robotics (cs.RO)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 11'],['/scholar?cites=1543039800876013962&as_sdt=20...,"['\n ', "" With the adoption of autonomou...","[array([], dtype=float64), array(['autonomous ..."
2,{'ObjectId': '639d9f0e6d67832e75895c48'},arxiv,['Towards Understanding Asynchronous Advantage...,"['Han Shen', 'Kaiqing Zhang', 'Mingyi Hong', '...","['\n ', ' Asynchronous and parallel impl...",['Machine Learning (cs.LG)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 10'],['/scholar?cites=17816434878891582694&as_sdt=2...,"['\n ', ' Asynchronous and parallel impl...","[array([], dtype=float64), array(['##ch', '##y..."
3,{'ObjectId': '639d9f0e6d67832e75895c49'},arxiv,['The Advantage Regret-Matching Actor-Critic'],"['Audrūnas Gruslys', 'Marc Lanctot', 'Rémi Mun...","['\n ', "" Regret minimization has played...",['Artificial Intelligence (cs.AI)'],https://scholar.google.com/scholar?q=advantage...,['Citado por 14'],['/scholar?cites=13622395690856603026&as_sdt=2...,"['\n ', "" Regret minimization has played...","[array([], dtype=float64), array(['##ability',..."
4,{'ObjectId': '639d9f0e6d67832e75895c4a'},arxiv,['Actor-Critic Sequence Training for Image Cap...,"['Li Zhang', 'Flood Sung', 'Feng Liu', 'Tao Xi...","['\n ', ' Generating natural language de...",['Computer Vision and Pattern Recognition (cs....,https://scholar.google.com/scholar?q=advantage...,['Citado por 111'],['/scholar?cites=16998376404483706528&as_sdt=2...,"['\n ', ' Generating natural language de...","[array([], dtype=float64), array(['captioning ..."


In [5]:
# Llenamos la base de datos.
# Se crean CONSTRAINTS para garantizar que los nodos no estén
# duplicados mientras configuramos algunos índices.
conn.query('CREATE CONSTRAINT papers IF NOT EXISTS FOR (p:Paper) REQUIRE p.id IS UNIQUE')
conn.query('CREATE CONSTRAINT fuentes IF NOT EXISTS FOR (f:Fuente) REQUIRE f.fuente IS UNIQUE')
conn.query('CREATE CONSTRAINT autores IF NOT EXISTS FOR (a:Autor) REQUIRE a.autor IS UNIQUE')
conn.query('CREATE CONSTRAINT categories IF NOT EXISTS FOR (c:Category) REQUIRE c.category IS UNIQUE')

[]

In [6]:
# query = '''
# WITH 'file:///totalPrueba.csv' AS data 
# LOAD CSV WITH HEADERS FROM data AS row
# MERGE (c:Category {category: row.categorias})
# '''
# conn.query(query, db='neo4j2')

query = '''
//Create and Relation Papers and Categories
LOAD CSV WITH HEADERS FROM 'file:///total_procesed.csv' AS row
MERGE (p:Paper {id:row._id}) ON CREATE SET p.titulo = row.titulo
WITH p, row
UNWIND split(row.clase_primaria, ', ') AS category_name
MERGE (c:Category {category: category_name})
MERGE (p)-[:IN_CATEGORY]->(c)
'''
conn.query(query,db='neo4j2')

[]

In [11]:
# query = '''
# //Create and Relation Papers and Categories
# LOAD CSV WITH HEADERS FROM 'file:///totalPrueba.csv' AS row
# MERGE (p:Paper {id:row.id}) ON CREATE SET p.titulo = row.titulo
# WITH p, row
# UNWIND split(row.categorias, ', ') AS category_name
# MERGE (c:Category {category: category_name})
# MERGE (p)-[:IN_CATEGORY]->(c)
# '''
# conn.query(query,db='neo4j2')

In [12]:
# query = '''
# WITH 'file:///totalPrueba.csv' AS data 
# LOAD CSV WITH HEADERS FROM data AS row
# MERGE (p:Paper {id:row.id}) ON CREATE SET p.titulo = row.titulo
# // connect categories
# WITH row, p
# UNWIND row.categorias AS category_name
# MATCH (p:Category {category: category_name})
# MERGE (p)-[:IN_CATEGORY]->(c)
# '''
# conn.query(query, db='neo4j2')

In [7]:
query = '''
//Create node for fuente and relationship for fuente and paper
WITH 'file:///total_procesed.csv' AS data 
LOAD CSV WITH HEADERS FROM data AS row
MERGE (f:Fuente {fuente:row.fuente})
WITH row, f
MATCH (p:Paper {id: row._id})
MERGE (p)-[:IN_LANGUAGE]->(f)
'''
conn.query(query,db='neo4j2')

[]

In [12]:
query = '''
//Create node for Autor and Relationship between papers and autores
LOAD CSV WITH HEADERS FROM 'file:///total_procesed.csv' AS row
UNWIND split(row.autores, ', ') AS autor_name
MERGE (a:Autor {name: autor_name})
WITH a, row
MATCH (p:Paper {id: row._id})
MERGE (a)-[:REALIZA]->(p)
'''
conn.query(query,db='neo4j2')

[]

![Alt text](GrafoEsquema.png)

### Consultas a BBDD Grafos

Ejemplo consulta a BBDD grafos

Consulta de Cypher utilizada: MATCH p=()-->() RETURN p LIMIT 100

![Alt text](EjemploConsultaGrafo.png)

In [ ]:
query = '''
MATCH (p:Paper)-[r:IN_CATEGORY]->(c:Category) 
RETURN p,c LIMIT 25
'''
conn.query(query, db='neo4j2')

In [15]:
query = '''
MATCH (p:Paper)-[r:IN_FUENTE]->(f:Fuente) 
RETURN f,p LIMIT 25
'''
conn.query(query, db='neo4j2')

[<Record l=<Node element_id='4:fd98e22d-4acd-4275-91fc-02d5dc9c7d3a:36238' labels=frozenset({'Language'}) properties={'language': 'en'}> p=<Node element_id='4:fd98e22d-4acd-4275-91fc-02d5dc9c7d3a:0' labels=frozenset({'Paper'}) properties={'titulo': 'The Comprehensive Model of\xa0Using In-Depth Consolidated Multimodal Learning to\xa0Study Trading Strategies in\xa0the\xa0Securities Market', 'id': 'doi:10.1007/978-3-031-16203-9_8'}>>,
 <Record l=<Node element_id='4:fd98e22d-4acd-4275-91fc-02d5dc9c7d3a:18176' labels=frozenset({'Language'}) properties={'language': 'en'}> p=<Node element_id='4:fd98e22d-4acd-4275-91fc-02d5dc9c7d3a:0' labels=frozenset({'Paper'}) properties={'titulo': 'The Comprehensive Model of\xa0Using In-Depth Consolidated Multimodal Learning to\xa0Study Trading Strategies in\xa0the\xa0Securities Market', 'id': 'doi:10.1007/978-3-031-16203-9_8'}>>,
 <Record l=<Node element_id='4:fd98e22d-4acd-4275-91fc-02d5dc9c7d3a:36239' labels=frozenset({'Language'}) properties={'language'

In [ ]:
query = '''
//Consulta comprobando si una categoria contiene 'Automative Engineering'
MATCH (c:Category)
WHERE c.category CONTAINS 'Automotive Engineering'
RETURN c;
'''
conn.query(query, db='neo4j2')

In [16]:
from pandas import DataFrame

query = '''
MATCH (p:Paper)
RETURN DISTINCT p.id, p.titulo, p.abstract
'''
dtf_data = DataFrame([dict(_) for _ in conn.query(query, db='neo4j')])
dtf_data.sample(10)

,p.id,p.titulo,p.abstract
2133,doi:10.1007/978-3-031-13433-3_1,What is Intelligent Construction?,None
7947,doi:10.1007/978-3-031-16865-9_12,Factors Influencing the Intention to Adopt Big...,None
12896,doi:10.1007/s10489-022-03190-3,AP-BERT: enhanced pre-trained model through av...,None
11493,doi:10.1038/s41598-022-24567-x,Detecting hierarchical organization of pervasi...,None
16238,doi:10.1007/978-3-031-11058-0_43,Simulation Model of Retarder Stopper on the En...,None
1761,doi:10.1007/978-981-19-4052-1_72,A Hybrid Gray Wolf Optimizer for Modeling and ...,None
419,doi:10.1007/978-3-031-19039-1_6,What Is on the Horizon?,None
365,doi:10.1007/978-981-19-3035-5_4,A Study on Reinforcement Learning-Based Traffi...,None
6176,doi:10.1007/978-981-19-4606-6_2,Instructions for the Preparation Intervention ...,None
2884,doi:10.1007/978-981-19-1457-7_19,Effect of Catalyst in the Pyrolysis of Waste P...,None
